In [1]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
import numpy as np
from PIL import Image
import io
import os
import os.path
import glob
from datetime import datetime
from google.cloud import vision
from google.cloud.vision import types

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bin/ROKS-78e082836307.json"

plt.figure(figsize=(50, 40))
plt.style.use('dark_background')

images = glob.glob('data/info/original/*.jpg')

<Figure size 3600x2880 with 0 Axes>

In [6]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in texts[0].bounding_poly.vertices])

    return texts[0].description

In [7]:
def change_to_number(s_power):
    list_of_numbers = re.findall(r'\d+', s_power)
    return int(''.join(list_of_numbers))

In [8]:
def change_to_list(chars):
    tmp_texts = list()
    s = ""
    for c in chars:
        if c != '\n':
            s += c
        else:
            tmp_texts.append(s)
            s = ""
    
    return tmp_texts

In [165]:
#crop & convert
start_time = datetime.now()

for image in images:
    initial = Image.open(image)
    width = initial.size[0]
    height = initial.size[1]

    path, file_name = os.path.split(image)

    #name
    crop_name = initial.crop((width*0.295, height*0.15, width*0.8, height*0.2))
    crop_name.save('data/info/croped/name/' + file_name)

    #kill
    crop_kill = initial.crop((width*0.56, height*0.28, width*0.72, height*0.44))

    lower_blue = (0, 0, 0)
    upper_blue =(255, 255, 60)

    img = np.array(crop_kill) 
    img = img[:, :, ::-1].copy() 

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_mask = cv2.inRange(img_hsv, lower_blue, upper_blue)
    img_result = cv2.bitwise_not(img, img)
    img_result1 = cv2.bitwise_and(img_result, img_result, mask=img_mask)
    
    cv2.imwrite('data/info/croped/kill/' + file_name, img_result1)
    
    crop_ds = initial.crop((width*0.20, height*0.23, width*0.8, height*0.98))
    crop_ds.save('data/info/croped/ds/' + file_name) 


end_time = datetime.now()

print (end_time - start_time)

0:00:17.665206


In [237]:
# Detect
start_time = datetime.now()

name = list()
power = list()
kill = list()
dead = list()
support = list()
t1 = list()
t2 = list()
t3 = list()
t4 = list()
t5 = list()

i = 0
for image in images:
    start_time_1 = datetime.now()
    path, file_name = os.path.split(image)
    
    # name
    detect_name = detect_text('data/info/croped_test/name/' + file_name)
    detect_name = change_to_list(detect_name)
    name.append(detect_name[0])
    power.append(change_to_number(detect_name[-2]))
    kill.append(change_to_number(detect_name[-1]))

    # kill
    detect_kill = detect_text('data/info/croped_test/kill/' + file_name)
    detect_kill = change_to_list(detect_kill)

    t1.append(change_to_number(detect_kill[0]))
    t2.append(change_to_number(detect_kill[1]))
    t3.append(change_to_number(detect_kill[2]))
    t4.append(change_to_number(detect_kill[3]))
    t5.append(change_to_number(detect_kill[4]))

    # ds
    detect_ds = detect_text('data/info/croped_test/ds/' + file_name)
    detect_ds = change_to_list(detect_ds)
    dead.append(change_to_number(detect_ds[detect_ds.index('전사')+1]))
    support.append(change_to_number(detect_ds[detect_ds.index('원조')+1]))
    
    end_time_1 = datetime.now()
    
    print(i, 'done... it took', (end_time_1 - start_time_1))
    i += 1
    
end_time = datetime.now()

print (end_time - start_time)

df = pd.DataFrame({'name': name,
                   'power':power,
                   'kill':kill,
                   'T1_kill':t1,
                   'T2_kill':t2,
                   'T3_kill':t3,
                   'T4_kill':t4,
                   'T5_kill':t5,
                   'dead':dead,
                   'support':support})

df.to_excel('data/info/result8.xlsx')

KeyboardInterrupt: 

In [227]:
tmp = list()

initial = Image.open('data/info/test.jpg')
# width = initial.size[0]
# height = initial.size[1]

# path, file_name = os.path.split(images[53])

# crop_kill = initial.crop((width*0.56, height*0.28, width*0.72, height*0.44))
img = cv2.imread('data/info/test2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

ret, img_thresh = cv2.threshold(
    img_blurred, 
    190, #name : 180
    255,
    cv2.THRESH_BINARY
)

cv2.imwrite('data/info/test4.jpg',img_thresh)


#lower_blue = (50, 50, 50)
#upper_blue =(255, 255, 255)

#img = np.array(initial) 
#img = img[:, :, ::-1].copy() 

#img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#img_mask = cv2.inRange(img_hsv, lower_blue, upper_blue)
#img_result = cv2.bitwise_not(img, img)

#cv2.imwrite('data/info/test4.jpg', img_result)

True

In [228]:
texts = detect_text('data/info/test4.jpg')
tmp.append(change_to_list(texts))

In [229]:
print(tmp)

[['청 미르', '전투력: 67,936,275', '처치: 21.585.021']]


In [138]:
print(images.index('data/info/original\Screenshot_20200727-203132_Rise of Kingdoms.jpg'))

10


In [231]:
# name

name = list()
power = list()
kill = list()
dead = list()
support = list()
t1 = list()
t2 = list()
t3 = list()
t4 = list()
t5 = list()

path, file_name = os.path.split(images[74])

detect_name = detect_text('data/info/croped_test/name/' + file_name)
detect_name = change_to_list(detect_name)
name.append(detect_name[0])
power.append(change_to_number(detect_name[1]))
kill.append(change_to_number(detect_name[2]))

# kill
# detect_kill = detect_text('data/info/croped_test/kill/' + file_name)
# detect_kill = change_to_list(detect_kill)

# t1.append(change_to_number(detect_kill[0]))
# t2.append(change_to_number(detect_kill[1]))
# t3.append(change_to_number(detect_kill[2]))
# t4.append(change_to_number(detect_kill[3]))
# t5.append(change_to_number(detect_kill[4]))

# # ds
# detect_ds = detect_text('data/info/croped_test/ds/' + file_name)
# detect_ds = change_to_list(detect_ds)
# dead.append(change_to_number(detect_ds[detect_ds.index('전사')+1]))
# support.append(change_to_number(detect_ds[detect_ds.index('원조')+1]))

# df = pd.DataFrame({'name': name,
#                    'power':power,
#                    'kill':kill,
#                    'T1_kill':t1,
#                    'T2_kill':t2,
#                    'T3_kill':t3,
#                    'T4_kill':t4,
#                    'T5_kill':t5,
#                    'dead':dead,
#                    'support':support})

ValueError: invalid literal for int() with base 10: ''

In [235]:
detect_name[-2]

'전투력: 49,277,194'